In [14]:
# === Imports === #
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yaml
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Load Configuration === #
def load_config(path):
    """Load configuration from YAML file."""
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Retrieve Account Information === #
account_info = mt5.account_info()
if account_info is None:
    print("Failed to retrieve account information.")
    mt5.shutdown()
    quit()

# === Extract Balance, Equity, and Floating Loss === #
balance = account_info.balance
equity = account_info.equity
floating_loss = account_info.profit

# === Display Results === #
print("=== Account Information ===")
print(f"Balance: {balance:.2f} USD")
print(f"Equity: {equity:.2f} USD")
print(f"Floating Loss: {floating_loss:.2f} USD")

# === Fetch and Process Running Trades === #
fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
df = fetcher.get_sheet_as_dataframe("Forex")
positions = mt5.positions_get()
df_positions = pd.DataFrame([pos._asdict() for pos in positions]) if positions else pd.DataFrame()

if df_positions.empty:
    print("No running trades found. Error:", mt5.last_error())
else:
    # Filter and rename columns
    df_positions = df_positions[["type", "volume", "price_open", "price_current", "sl", "tp", "profit", "symbol"]]
    df_positions["type"] = df_positions["type"].replace({0: "Buy", 1: "Sell"})
    df_positions.rename(columns={
        "symbol": "Symbol",
        "type": "Action",
        "volume": "Lot",
        "price_open": "Price",
        "price_current": "Price Current",
        "sl": "SL",
        "tp": "TP",
        "profit": "Profit"
    }, inplace=True)

    # Aggregate by Symbol and Action
    def aggregate_group(group):
        symbol = group["Symbol"].iloc[0]
        action = group["Action"].iloc[0]
        lot = group["Lot"].sum()
        warnings = []

        # === Lot mismatch check === #
        df_lot_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[5]].sum()
        if not pd.isna(df_lot_value) and round(df_lot_value, 2) > round(lot, 2):
            warnings.append(f"Lot Mismatch: Expected {round(df_lot_value, 2)}, Found {round(lot, 2)}")

        # === SL/TP mismatch check === #
        df_sl_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[7]].sum()
        sl_mode = group["SL"].mode().iloc[0] if not group["SL"].mode().empty else np.nan
        if group["SL"].nunique() > 1 or (df_sl_value != 0 and round(df_sl_value, 2) != round(sl_mode, 2)):
            warnings.append("SL Mismatch")

        df_tp_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[8]].sum()
        tp_mode = group["TP"].mode().iloc[0] if not group["TP"].mode().empty else np.nan
        if group["TP"].nunique() > 1 or round(df_tp_value, 2) != round(tp_mode, 2):
            warnings.append("TP Mismatch")

        # Combine warnings
        warning_message = "; ".join(warnings) if warnings else np.nan

        return pd.Series({
            "Lot": lot,
            "Price": (group["Price"] * group["Lot"]).sum() / group["Lot"].sum(),
            "Price Current": group["Price Current"].mean(),
            "SL": group["SL"].mode().iloc[0] if not group["SL"].empty else np.nan,
            "TP": group["TP"].mode().iloc[0] if not group["TP"].empty else np.nan,
            "Profit": group["Profit"].sum(),
            "Warning": warning_message
        })

    df_positions = (
        df_positions.groupby(["Symbol", "Action"], as_index=False)
        .apply(lambda group: aggregate_group(group))
        .reset_index(drop=True)
    )

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Calculate Coefficients === #
df_positions["Coeff"] = np.abs(df_positions["Profit"] / (df_positions["Price Current"] - df_positions["Price"])) / (df_positions["Lot"] * 100)
df_positions["Pips"] = np.abs(df_positions["TP"] - df_positions["SL"])

# Merging the Type column based on Symbol
df_type = df[[df.columns[0], df.columns[1]]].drop_duplicates()
df_type.columns = ["Symbol", "Type"]
df_positions = pd.merge(df_positions, df_type, on="Symbol", how="left")

# Handling missing types (optional)
df_positions['Type'].fillna("Unknown", inplace=True)

# === Update Coefficients in Config (Exclude NaN) === #
df_avg_coeff = df_positions.groupby("Symbol")["Coeff"].mean().reset_index()
differences = {}

for _, row in df_avg_coeff.iterrows():
    symbol = row["Symbol"]
    avg_coeff = round(row["Coeff"], 5)
    
    # Only update if coefficient is not NaN
    if not pd.isna(avg_coeff):
        if symbol in config.get("symbol_coefficients", {}):
            old_coeff = config["symbol_coefficients"].get(symbol, avg_coeff)
            if old_coeff != avg_coeff:
                differences[symbol] = {"Old": old_coeff, "New": avg_coeff}
        
        # Add or update the coefficient
        config["symbol_coefficients"][symbol] = avg_coeff

# === Save Updated Configuration === #
with open(CONFIG_PATH, "w") as file:
    yaml.safe_dump(config, file)

# === Display Changed Coefficients === #
print("Changed symbol_coefficients:")
for symbol, change in differences.items():
    print(f"{symbol}: {change['Old']} -> {change['New']}")

# Creating new rows for Balance, Equity, and Floating Loss
df_balance = pd.DataFrame([
    {"Symbol": "Balance", "Price": balance},
    {"Symbol": "Equity", "Price": equity},
    {"Symbol": "Floating Loss", "Price": floating_loss}
])

# Inserting the new rows at the top of the DataFrame
df_final = pd.concat([df_balance, df_positions], ignore_index=True)

# === Uploade Updated Dataframe === #
uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
uploader.upload_dataframe(df_final, "Forex Summary", replace=True)

# === Display Final Merged DataFrame === #
print("Final Merged DataFrame:")
df_final

=== Account Information ===
Balance: 4507.29 USD
Equity: 4328.67 USD
Floating Loss: -1427.50 USD
✅ Successfully retrieved data from 'Forex' as a DataFrame.


C:\Users\syahr\AppData\Local\Temp\ipykernel_512\3565058515.py:106: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: aggregate_group(group))
C:\Users\syahr\AppData\Local\Temp\ipykernel_512\3565058515.py:123: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation in

Changed symbol_coefficients:
ADS.XE: 1.00038 -> 1.00036
AI.EPA: 1.11978 -> 1.12052
AIR.EPA: 1.12155 -> 1.11823
ALGOUSD: 24.97366 -> 25.0
ATOMUSD: 2.50484 -> 2.5102
AUDCAD: 718.36057 -> 718.30406
AUDCHF: 1191.35447 -> 1192.46145
AUDJPY: 6.80816 -> 6.97218
AUDNZD: 593.53337 -> 594.41377
AVAXUSD: 0.99773 -> 0.99806
AZN.LSE: 1.33028 -> 1.33125
BARC.LSE: 1.33098 -> 1.33409
BATS.LSE: 1.32959 -> 1.33083
BBVA.BM: 1.12085 -> 1.11916
BCHUSD: 0.10003 -> 0.09989
BEI.XE: 1.00083 -> 1.00074
BMW.XE: 1.00484 -> 1.00442
BNBUSD: 0.10034 -> 0.10007
BNP.EPA: 1.11511 -> 1.11229
CADCHF: 1190.91847 -> 1192.77108
CADJPY: 6.7963 -> 6.80189
CHFJPY: 6.79121 -> 6.78873
CSL.ASX: 0.64765 -> 0.64808
DKI.TSE: 0.0068 -> 0.00681
DOTUSD: 2.50678 -> 2.5
ENEL.MIL: 1.11924 -> 1.12173
ENI.MIL: 1.10714 -> 1.11765
ETCUSD: 2.5 -> 2.54237
EURAUD: 647.82663 -> 649.71141
EURCAD: 718.39454 -> 717.53358
EURCHF: 1191.67579 -> 1192.92605
EURGBP: 1330.49861 -> 1333.57432
EURJPY: 6.85153 -> 6.7927
EURMXN: 51.58009 -> 51.58337
EURNZD: 5

,Symbol,Price,Action,Lot,Price Current,SL,TP,Profit,Warning,Coeff,Pips,Type
0,Balance,4507.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Equity,4328.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Floating Loss,-1427.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL.NAS,213.670,Buy,0.01,212.800,67.430,217.470,-0.87,SL Mismatch,1.0,150.040,Stock
4,AAVEUSD,217.270,Buy,0.01,233.510,0.010,239.780,8.12,NaN,0.5,239.770,Crypto
...,...,...,...,...,...,...,...,...,...,...,...,...
185,XNGUSD,3.942,Sell,0.01,3.875,7.923,3.837,6.70,NaN,100.0,4.086,Forex
186,XOM.NYSE,113.935,Buy,0.02,109.340,65.060,115.250,-9.19,SL Mismatch,1.0,50.190,Stock
187,XTIUSD,62.720,Buy,0.01,63.030,0.010,63.840,3.10,NaN,10.0,63.830,Forex
188,XTIUSD,61.570,Sell,0.01,63.150,124.300,60.470,-15.80,NaN,10.0,63.830,Forex


In [ ]:
import time
import MetaTrader5 as mt5
import pandas as pd

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# Load DataFrame from Google Sheets
df = fetcher.get_sheet_as_dataframe("Forex")
df.columns = df.iloc[5]
df = df.iloc[6:-3].reset_index(drop=True)

# Initialize counters for success and failure
success_count = 0
failure_count = 0

# Fetch unique symbols from running trades in MT5
positions = mt5.positions_get()
if positions is None or len(positions) == 0:
    print("No running trades found. Exiting.")
    mt5.shutdown()
    quit()

# Get unique symbols from MT5 positions
symbol_list = sorted(list(set(pos.symbol for pos in positions)))

# Convert positions to DataFrame for easy filtering
df_positions = pd.DataFrame([pos._asdict() for pos in positions])

# Iterate through each unique symbol from MT5 positions
for symbol in symbol_list:
    for action in ["Buy", "Sell"]:
        # Filter positions for the current symbol and action
        action_type = 0 if action == "Buy" else 1
        filtered_positions = df_positions[(df_positions['symbol'] == symbol) & (df_positions['type'] == action_type)]
        
        if filtered_positions.empty:
            print(f"Symbol: {symbol}\nAction: {action}\n⚠️ No {action} positions running. Skipping.")
            continue

        # Filter rows for the current symbol and action from Google Sheets
        df_filtered = df[(df["Symbol"] == symbol) & (df["Action"] == action)]
        
        if df_filtered.empty:
            print(f"Symbol: {symbol}\nAction: {action}\n⚠️ No matching TP/SL settings in Google Sheets.")
            continue
        
        tp_value = round(df_filtered['TP'].values[0], 5)
        sl_value = round(df_filtered['SL'].values[0], 5)

        # Iterate over filtered positions for modification
        for _, pos in filtered_positions.iterrows():
            order_id = pos['ticket']
            current_tp = pos['tp']
            current_sl = pos['sl']

            # Only modify if the new TP/SL values differ from current
            if current_tp == tp_value and current_sl == sl_value:
                print(f"Symbol: {symbol}\nAction: {action}\n✅ No changes for {action} position {order_id}. TP/SL already set.")
                continue

            # Modify TP/SL using position ID
            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": order_id,
                "sl": sl_value,
                "tp": tp_value
            }

            # Send modification request
            result = mt5.order_send(request)
            #time.sleep(0.5)  # Delay of 0.5 second between each request

            # Check if result is None (Error)
            if result is None:
                print(f"Symbol: {symbol}\nAction: {action}\n⚠️ Failed to modify {action} position {order_id}. Error: Request returned None.")
                failure_count += 1
                continue

            # Check if the modification was successful
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"Symbol: {symbol}\nAction: {action}\n✅ {action} position {order_id} modified successfully.")
                print(f"Before: TP={current_tp}, SL={current_sl}")
                print(f"After:  TP={tp_value}, SL={sl_value}\n")
                success_count += 1
            else:
                print(f"Symbol: {symbol}\nAction: {action}\n⚠️ Failed to modify {action} position {order_id}. Error: {result.comment}")
                failure_count += 1

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Summary Report ===
print("\n=== TP/SL Modification Summary ===")
print(f"Total Successful Modifications: {success_count}")
print(f"Total Failed Modifications: {failure_count}")